In [1]:
import sys
import os
from pathlib import Path

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

GNDTRUTH = Path('../data/raw/COMMON_images_masks/')
RAWIMGS = Path('../data/raw/GROUP_images/')
DATA = (RAWIMGS.parent).parent
MASKS = DATA/'masks'

import SimpleITK as sitk
from utils import *
from linear_registration import register_linear

In [2]:
mov_names = ['g1_54', 'g1_55', 'g1_56']
fix_name = 'common_40'

fix_im = sitk.ReadImage(GNDTRUTH/(fix_name+'_image.nii.gz'))
mov_ims = [sitk.ReadImage(RAWIMGS/(name+'_image.nii.gz')) for name in mov_names]
fix_msk = sitk.ReadImage(MASKS/(fix_name+'_regmask.nii.gz'))
mov_msks = [sitk.ReadImage(MASKS/(name+'_mask.nii.gz')) for name in mov_names]

# Arrays from images
fix_im_data = sitk.GetArrayFromImage(fix_im)
fix_msk_data = sitk.GetArrayFromImage(fix_msk)
mov_ims_data = [sitk.GetArrayFromImage(img) for img in mov_ims]
mov_msks_data = [sitk.GetArrayFromImage(msk) for msk in mov_msks]


In [3]:
registrations = []
for idx, (mov_im, mov_msk) in enumerate(zip(mov_ims, mov_msks)):
    print(f'---- Image {idx} ----')
    registrations.append(register_linear(fix_im, mov_im, fix_mask=None, mov_mask=mov_msk, verbose=True))

---- Image 0 ----
Creating bone mask for fixed image...
Generating mask from HU thresholding with HU_min=185, HU_max=500...
Mask generated!
Cropping fixed image to ROI...
Cropping moving image to ROI...
Estimating affine transformation...
Final transform:
itk::simple::CompositeTransform
 CompositeTransform (0x678ba50)
   RTTI typeinfo:   itk::CompositeTransform<double, 3u>
   Reference Count: 1
   Modified Time: 1519184
   Debug: Off
   Object Name: 
   Observers: 
     none
   TransformQueue: 
   >>>>>>>>>
   AffineTransform (0x674ec90)
     RTTI typeinfo:   itk::AffineTransform<double, 3u>
     Reference Count: 1
     Modified Time: 1519030
     Debug: Off
     Object Name: 
     Observers: 
       none
     Matrix: 
       1.02954 0.139173 -0.0653332 
       -0.130217 0.788417 -0.0463344 
       0.0261473 -0.11817 0.93616 
     Offset: [52.9403, -4.91447, -1560.77]
     Center: [-79.3516, -127.672, 842.698]
     Translation: [-22.2284, -6.61421, -1601.56]
     Inverse: 
       0.949

In [4]:
mov_ims_reg = [reg['registered_image'] for reg in registrations]
mov_msks_reg = [reg['registered_mask'] for reg in registrations]
lin_tfms = [reg['affine_tfm'] for reg in registrations]

In [5]:
mov_ims_reg_data = [sitk.GetArrayFromImage(im) for im in mov_ims_reg]
mov_msks_reg_data = [sitk.GetArrayFromImage(im) for im in mov_msks_reg]

In [9]:
show_interactive_overlay(fix_im_data, mov_ims_reg_data[2], 'coronal')

interactive(children=(IntSlider(value=256, description='slc', max=511), Output()), _dom_classes=('widget-inter…

<function utils.visualization.show_coronal_overlay(fix_arr, mov_arr, slc)>

In [92]:
# Change folder to 'with_regmask' if the fix_mask is provided in the registration. Otherwise use folder 'with_HUmask'
for idx, name in enumerate(mov_names):
    sitk.WriteImage(mov_ims_reg[idx], DATA/('linear_registrations/with_HUmask/'+name+'_linreg_'+fix_name+'_img.nii.gz'))
    sitk.WriteImage(mov_msks_reg[idx], DATA/('linear_registrations/with_HUmask/'+name+'_linreg_'+fix_name+'_msk.nii.gz'))
    sitk.WriteTransform(lin_tfms[idx], DATA/('linear_registrations/with_HUmask/transforms/'+name+'_linreg_'+fix_name+'.tfm'))

In [ ]:
fix_im2 = sitk.ReadImage(GNDTRUTH/'common_41_image.nii.gz')
fix_im3 = sitk.ReadImage(GNDTRUTH/'common_42_image.nii.gz')
fix_im2_data = sitk.GetArrayFromImage(fix_im2)
fix_im3 = sitk.rea